In [ ]:
# WKNN

import torch
import numpy as np
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import tensorflow as tf
import time
import operator

# Define the data structure
class MyDataset(Dataset):
    def __init__(self,filepath):
        xy = np.loadtxt(filepath,delimiter=',',dtype=np.float32)
        self.len = xy.shape[0]
        
        # load the data after the second column of the dataset (RSS data)
        self.x_data = torch.from_numpy(xy[:,2:])
        # load the first two columns of the dataset (Coordinate)
        self.y_data = torch.from_numpy(xy[:,0:2])
        
        #print("The data has prepared...")
        
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.len

# Initialize the positioning error
error_0 = []

for z in range(1):
    print(z)
    # Settings
    batch_size = 14
    
    # load traning and testing dataset
    train_file = "./dataset/03_14_train.csv"
    test_file = "./dataset/03_10_test.csv"
    mydataset_train = MyDataset(train_file)
    mydataset_test = MyDataset(test_file)
    train_loader = DataLoader(dataset=mydataset_train,
                             batch_size=batch_size,
                             shuffle=True,
                             num_workers=2)
    test_loader = DataLoader(dataset=mydataset_test,
                             batch_size=10,
                             shuffle=True,
                             num_workers=2)

    start_time = time.time()
    
    # Data preparing
    for data, labels in train_loader:
        train_data = data.view(-1, 64).numpy()
        train_crd = labels.view(-1,2).numpy()
        train_crd = 30*train_crd
    for data, labels in test_loader:
        test_data = data.view(-1, 64).numpy() 
        test_crd = labels.view(-1,2).numpy()
        test_crd = 30*test_crd

    # 1.Define the function to calculate the Euclidean distance
    def euclideanDistance(instance1,instance2,length):
        dis0 = 0
        for i in range(0,length):
            dis0 += (instance1[i]-instance2[i])**2
        return dis0**0.5
    
    # 2.Select the K RPs with the shortest distance from the test point
    def getNeighbors(trainSet,testPoint,k):
        distance = []
        length = len(testPoint)-2
        for i in range(0,len(trainSet)):
            dis1 = euclideanDistance(testPoint,trainSet[i],length)
            distance.append((trainSet[i],dis1))
        distance.sort(key = operator.itemgetter(1))      ## 对距离按照第二个域排序
        neighbors = []
        for j in range(0,k):
            neighbors.append(distance[j])
        return neighbors
    
    # 3.Calculate the weighted average of the K RP coordinates
    def getResponse(neighbors):
        add_weight = 0
        crd = 0
        for i in range(0,len(neighbors)):
            add_weight += 1/neighbors[i][1] 
        for j in range(0,len(neighbors)):
            crd += (neighbors[j][0][64:66]/neighbors[j][1])/add_weight
        return crd
    
    # 4.Define the accuracy function
    def getAccuracy(pred_crd,test_crd):
        return np.mean(np.sqrt(np.sum((pred_crd-test_crd)**2,1)))
    
    # 5.main
    train_file = np.hstack((train_data,train_crd))
    test_file = np.hstack((test_data,test_crd))
    k = 4
    pred_crd = []
    for i in range(0,len(test_file)):
        neighbors = getNeighbors(train_file, test_file[i], k)
        crd = getResponse(neighbors)
        pred_crd.append(crd)
    accuracy = getAccuracy(pred_crd,test_crd)
    print("accuracy:" + repr(accuracy) + "m")               ## print()内输出的内容格式必须一致
    print("--- Run time: %s mins ---" % np.round(((time.time() - start_time)/60),4))
    
    #error_0.append(e)
